<img src="https://github.com/YongBeomKim/nltk_basic/raw/master/data/cosin.png" width=800>
<br/>
# **Chapter 4 | 추천시스템**
참고사이트 : https://www.machinelearningplus.com/nlp/cosine-similarity/
## **1 데이터 불러오기**

In [1]:
! apt-get update
! apt-get install g++ openjdk-8-jdk 
! pip3  install  nltk konlpy matplotlib gensim 

! apt-get install fonts-nanum-eco
! apt-get install fontconfig
! fc-cache -fv
! cp /usr/share/fonts/truetype/nanum/Nanum* /usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data/fonts/ttf/
! rm -rf /content/.cache/matplotlib/*

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [564 B]
Get:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [819 B]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [113 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Packages [19.8 kB]
Get:11 http://archive.ubuntu.com/u

In [2]:
import nltk
nltk.download('wordnet')

import pandas as pd
import io, requests
url = "https://raw.githubusercontent.com/YongBeomKim/nltk_basic/master/data/movies_metadata.csv"
response = requests.get(url).content
movies = pd.read_csv(io.StringIO(response.decode('utf-8')),
                    usecols=['original_title', 'overview', 'title'], low_memory=False)
movies = movies.dropna(axis=0)
movies.shape

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


(44506, 3)

In [3]:
movie_plot_li = movies['overview']
movie_info_li = movies['title']
movies.head(3)

,original_title,overview,title
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Toy Story
1,Jumanji,When siblings Judy and Peter discover an encha...,Jumanji
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,Grumpier Old Men


# 텍스트 전처리

In [0]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

class LemmaTokenizer(object):
  def __init__(self):
    self.wnl = WordNetLemmatizer()
    self.tokenizer = RegexpTokenizer('(?u)[A-z]+')
  def __call__(self,doc):
    return([self.wnl.lemmatize(t) for t in self.tokenizer.tokenize(doc)])

In [8]:
# 사이킷런에 위에서 정의한 토크나이저를 입력으로 넣는다
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 3, tokenizer=LemmaTokenizer(), stop_words = 'english')
X = vectorizer.fit_transform(movie_plot_li[:10000])
vocabulary = vectorizer.get_feature_names()

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [9]:
from sklearn.metrics.pairwise import cosine_similarity
movie_sim = cosine_similarity(X)
print(movie_sim.shape)
movie_sim

(10000, 10000)


array([[1.        , 0.02930725, 0.00580921, ..., 0.01088994, 0.01071711,
        0.        ],
       [0.02930725, 1.        , 0.05286611, ..., 0.00904565, 0.00890209,
        0.        ],
       [0.00580921, 0.05286611, 1.        , ..., 0.00466609, 0.00459204,
        0.        ],
       ...,
       [0.01088994, 0.00904565, 0.00466609, ..., 1.        , 0.00860824,
        0.        ],
       [0.01071711, 0.00890209, 0.00459204, ..., 0.00860824, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [0]:
#특정 영화와 유사한 영화목록 츌력하기
def similar_recommend_by_movie_id(movielens_id,rank=8):
  movie_index = movielens_id -1
  similar_movies = sorted(list(enumerate(movie_sim[movie_index])),key=lambda x:x[1],reverse=True)
  print("-----{} : 관람객 추천영화 -----".format(movie_info_li[similar_movies[0][0]]))
  
  for no, movie_idx in enumerate(similar_movies[1:rank]):
    print("추천영화 {}순위: {}".format(no,movie_info_li[movie_idx[0]]))

In [20]:
similar_recommend_by_movie_id(1,rank=10)

-----Toy Story : 관람객 추천영화 -----
추천영화 0순위: My Man Godfrey
추천영화 1순위: The Hunchback of Notre Dame
추천영화 2순위: Platoon
추천영화 3순위: Vernon, Florida
추천영화 4순위: Lolita
추천영화 5순위: Stagecoach
추천영화 6순위: M. Butterfly
추천영화 7순위: Bicentennial Man
추천영화 8순위: Faces of Death II
